In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the data
data = np.loadtxt("./data.txt")

# Create the design matrix X
X = np.column_stack([
    data[:, 0]**2,          # x^2
    data[:, 0]*data[:, 1],  # xy
    data[:, 1]**2,          # y^2
    data[:, 0],             # x
    data[:, 1]              # y
])

bVec = -np.ones(len(data))
pVec = np.linalg.solve(X.T @ X, -X.T @ bVec)  

# Unpack pVec into parameters
A, B, C, D, E = pVec
print(f'Parameters: A={A}, B={B}, C={C}, D={D}, E={E}')

# Eigenvalue matrix
M = np.array([[A, B/2], [B/2, C]])
eigenvalues, eigenvectors = np.linalg.eig(M)

# Semi-axes lengths
a = np.sqrt(1 / np.abs(eigenvalues[0]))
b = np.sqrt(1 / np.abs(eigenvalues[1]))

# Ellipse orientation
angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))

# Ellipse center
centerX = -D / (2 * A)
centerY = -E / (2 * C)

# Plotting
ellipse = patches.Ellipse([centerX, centerY], 2*a, 2*b, angle=angle, fill=False, color='red', label='Fitted Ellipse')
plt.gca().add_patch(ellipse)
plt.scatter(data[:, 0], data[:, 1], label='Data Points')
plt.legend()
plt.axis('equal')
plt.show()